# Auto-STED for imaging of FISH spot pairs

## How-To run:

1) Run the cell bellow ONCE if you (re)started the notebook

(to run a cell, click in it and press Ctrl-Enter)

In [1]:
from specpy import *
from Util.coordinates import *
from Util.datastructures import *
from Util.tile_util import *
from Util.imspector_util import *
from Util.dot_detection.Fiji_coordinate_detection import *
from Util.coordinate_util import *
from Util.dot_detection.Fiji_dummy import *
from Util.log_util import *
import pprint
from Util import datastructures, imspector_util
import time
from matplotlib import pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from scipy import ndimage, spatial, stats
from spot_util import *
from display_util import *
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

class FinishedException(Exception):
    def __init__(self, n_images, time_passed, *args):
        self.n_images = n_images
        self.time_passed = time_passed
        super(FinishedException, self).__init__(n_images, time_passed, *args)
        
def get_relative_coords(current_coords, coords):
    '''
    make all coordinates in coords relative to current_coords
    '''
    res = list()
    for c in coords:
        res.append(list(np.array(c, dtype=float) - np.array(current_coords,  dtype=float)))
    return res

def do_runs(outpaths, coords, max_time, max_sted_images, logname='log.csv',  sted_settings = 'C:/Users//RESOLFT/Desktop/det.json'):
    
    if len(outpaths) != len(coords):
        print('please specify an output path for every coordinate')
        return    
        
    for i in range(len(coords)):
        do_run(outpaths[i], coords[i], max_time=max_time, max_sted_images=max_sted_images,
               first_z_size=2e-5, logname=logname, sted_settings=sted_settings)

def get_current_stage_coords(im = None):
    
    if im is None:
        im = Imspector()
        
    im.create_measurement()
    ms = im.active_measurement()
    
    coords = [ms.parameters('ExpControl/scan/range/offsets/coarse/'+ c + '/g_off') for c in 'xyz']
    
    im.close(ms)
    
    return coords

def do_run(outpath, start_coords, move_size = [6e-5, 6e-5], fov_size_ov = [5e-05, 5e-05, 1e-5], first_z_size= None,
           fov_size_sted = [3e-06, 3e-06, 1.4e-06], max_time = None, max_sted_images = None,
           ov_settings = 'C:/Users//RESOLFT/Desktop/ov013.json', sted_settings = 'C:/Users//RESOLFT/Desktop/det013.json',
           logname='log.csv'):
    
    # keep track of time and number of STED images taken
    start_time = time.time()
    n_sted_images = 0
    
    # connect
    im = Imspector()

    # 
    settings = Settings()
    settings.load_from_file(ov_settings)

    # coordinate generator (offset = 0)
    coords = df_circle_generator(move_size)

    #keep track of maximum z -> use that as next midplane
    zpos = 0.0;
    
    # setup output dir and name management
    d = os.path.dirname(outpath)
    if not os.path.exists(d):
        os.makedirs(d)
        
    # setup logging
    logfd = open(os.path.join(outpath, logname), 'w')
    logwriter = setup_csv_writer(logfd)
        
    nm = datastructures.NameManagement(d)
    nm.add_counter('field')
    nm.add_counter('sted')

    # measurement will be created on first run
    ms = None    
    first_field = True

    
    for c in coords:

        try:
            
            # set the correct fov
            c.set_fov_len(fov_size_ov)
            
            # do a larger z stack at the first field
            if (first_z_size != None) and first_field:
                first_field = False
                fov_ = c.get_fov_len()
                fov_[2] = first_z_size
                c.set_fov_len(fov_)


            
            # set stage offset
            stage_old = c.get_bench_coords()
            stage_new = list(np.array(stage_old) + np.array(start_coords))
            c.set_bench_coords(stage_new)
            
            # z-correction
            offset_old = c.get_scan_offset()
            offset_old[2] = zpos
            c.set_offset_coords(offset_old)

            settings.set_to_coordinates(c)

            # acquire image and save
            name = nm.get_next_image_name('field')
            acquire_measurement(im, settings, ms)
            ms = im.active_measurement()
            ms.save_as(name)
            
            # log the acquisition
            
            stg_coords_for_log = list(np.array(c.get_bench_coords(), dtype=float))
            l = make_csv_line(name, stg_coords=stg_coords_for_log, scan_coords=c.get_scan_offset())
            logwriter.writerow(l)

            
            # find pairs
            sites = pair_finder(ms, thresholds = [0.9, 0.7], normalize=False)

            # dummy detection
            #sites = [[10, 10, 5]]
            
            # get images in both channels
            # and plot the detections in max-projection
            stack1 = ms.stack(0).data()[0,:,:,:]
            stack2 = ms.stack(1).data()[0,:,:,:]
            draw_detections_2c(stack1, stack2, [s[-1::-1] for s in sites], [1, 10], 0, 3)
            plt.show()

            pixelsd = get_pixel_size(ms)
            corner = middle2corner(c.get_scan_offset(), c.get_fov_len(), pixelsd)
            actual_coords = corner2spot(corner, sites, pixelsd)

              
            # check if we should stop
            time_passed = time.time() - start_time
                    
            ## check if time has run out
            if (max_time != None) and (time_passed > max_time):
                raise FinishedException(n_sted_images, time_passed)           


            # update focus
            zpos += focus_in_stack(ms.stack(0).data(), pixelsd[2])
            
            # inform about new focus and detected spots
            print('new z:' + str(zpos))
            print(actual_coords)

            co = c.copy()
            settings2 = Settings()
            settings2.load_from_file(sted_settings)

            # restart counting the sted images ( -> field1sted0 - filed1stedn, field2sted0 - field2stedn, ...)
            nm.reset_counter('sted')

            for i in actual_coords:
                
                # set the fov and spot coordinates
                co.set_fov_len(fov_size_sted)
                co.set_offset_coords(i)
                settings2.set_to_coordinates(co)

                # do measurement and save
                name = nm.get_next_image_name('sted')
                acquire_measurement(im, settings2, ms)
                ms = im.active_measurement()
                ms.save_as(name)  
                
                # log the acquisition
                stg_coords_for_log = list(np.array(c.get_bench_coords(), dtype=float))
                l = make_csv_line(name, stg_coords=stg_coords_for_log, type='sted', scan_coords=list(co.get_scan_offset()))
                logwriter.writerow(l)
                
                # check if we should stop
                time_passed = time.time() - start_time
                n_sted_images += 1
                
                ## check if we have enough images
                if (max_sted_images != None) and (n_sted_images >= max_sted_images):
                    raise FinishedException(n_sted_images, time_passed)
                    
                ## check if time has run out
                if (max_time != None) and (time_passed > max_time):
                    raise FinishedException(n_sted_images, time_passed)

                    
        except FinishedException as e:
            print('RUN FINISHED. Acquired ' + str(e.n_images) + ' STED images in ' + str(e.time_passed) + 's.')
            logfd.close()
            break
            
        except KeyboardInterrupt:
            print('RUN STOPPED BY USER')
            logfd.close()
            break


# Simple acquisition scheduling GUI

Run the code below to display a simple `ipywidgets` GUI, allowing you to enqueue acquisitions and run them.

Basically, you only need to do the following steps:

1) run the cell below, a GUI will be displayed

2) move the microscope stage to the position you want to image at

3) enter the path of where to save data (folder path ending with a `/`) as well as maximum time and maximum amount of images 
4) repeat 3) for multiple measurements on one slide (note that you have to set different save paths for each acquisition)
5) click `Run` to start the measurement

If you make a mistake in setting up the measurements, you can clear the measurement list by clicking the corresponding button. In that case, you have to enqueue all measurements again, however.

## Stopping a run

To stop a measurement run, you have to do the following:

1) click the 'STOP' button at the top of this page

2) click the 'PAUSE' button in Imspector to end the currently running acquisition

3) If there are multiple measurements in your queue, this will only abort the current one - you have to repeat the above steps for each of them.

In [4]:
import time

from ipywidgets import widgets
from IPython.display import display, clear_output

worklist = list()

txt_path = widgets.Text('C:/Users/RESOLFT/Desktop/AUTOMATION/TEST', description = 'save path', layout=widgets.Layout(width='100%'))
flt_maxhrs = widgets.FloatText(12.0, description = 'maximum time (hours)')
int_maximgs = widgets.IntText(500, description = 'maximum images')
display(txt_path)
display(flt_maxhrs)
display(int_maximgs)


btn_add = widgets.Button(description = 'Add measurement')
btn_clear = widgets.Button(description = 'Clear measurement list')
btn_run = widgets.Button(description = 'Run')
hbox_btns = widgets.HBox([btn_add, btn_clear, btn_run])
display(hbox_btns)

def onclick_add(btn):
    
    clear_output()
    if txt_path.value in [s['path'] for s in worklist]:
        print('WARNING: the save path is already used for another scheduled acquisition, please change it')
    else:
        worklist.append({'path' : txt_path.value, 'time' : flt_maxhrs.value, 'maximgs' : int_maximgs.value,
                        'coords' : get_current_stage_coords()})
        
    print('------------ SCHEDULED ACQUISITIONS -------------')
    for wli in worklist:
        print ('at coordinates {} for {} hours or a maximum of {} images \n will be saved to {}'.format(
                *list(map(str, [wli['coords'], wli['time'], wli['maximgs'], wli['path']]))))
        
btn_add.on_click(onclick_add)

def onclick_clear(btn):
    clear_output()
    worklist.clear()
    print('WORKLIST CLEARED')
    
    
btn_clear.on_click(onclick_clear)

def onclick_run(btn):
    
    clear_output()
    
    for wli in reversed(worklist):
        print(time.strftime('%Y-%m-%d_%H:%M:%S', time.localtime()) + 
              ' -- started measurement at x: {} y: {} z: {}'.format(*list(map(str, wli['coords']))))
        
        do_run(wli['path'], wli['coords'], first_z_size=2e-5,
               max_time=wli['time'] * 60.0 * 60.0, max_sted_images=wli['maximgs'])  #,
               #sted_settings = 'C:/Users//RESOLFT/Desktop/det013_30sted_water.json',
               #v_settings = 'C:/Users//RESOLFT/Desktop/ov013_30sted_water.json')
               #sted_settings = 'C:/Users//RESOLFT/Desktop/det013_503d_45sted.json',
               #fov_size_sted = [1e-06, 1e-06, 1e-06])
        
        print(time.strftime('%Y-%m-%d_%H:%M:%S', time.localtime()) + 
              ' -- finished measurement, data saved to ' + wli['path'])
        
    worklist.clear()
    
btn_run.on_click(onclick_run)

2018-07-05_19:04:20 -- started measurement at x: 0.00111802 y: 0.00067126 z: 0.00186105


# --- OLD CODE ---


# OPTION 1: Measurement at current position 
2) output path in the cell below
3) run the cell (Imspector has to be running and Tools->Run server has to be ckecked in Imspector)

# Stopping the run
4) click the "STOP"-button at the top of this window
5) the current measurement in Imspector will finish, but won't be saved

In [ ]:
# where to save results
outpath = 'C:/Users//RESOLFT/Desktop/AUTOMATION/TEST/'

# the maximum number of sted images to acquire
max_sted_images = 500

# the maximum time to spend in one area (in SECONDS)
max_time = 6 * 60 * 60 # 12hrs

do_run(outpath, get_current_stage_coords(), max_time=max_time, max_sted_images=max_sted_images, sted_settings = 'C:/Users//RESOLFT/Desktop/det013.json')


# OPTION 2: Measurement at multiple coordinates
2) insert coordinates and result paths in the cell below
3) run the cell (Imspector has to be running and Tools->Run server has to be ckecked in Imspector)

# Stopping the run
4) click the "STOP"-button at the top of this window
5) the current measurement in Imspector will finish, but won't be saved

In [ ]:
Imspector().active_measurement().parameters('')

In [ ]:
# GLOBAL stage coordinates of where to do scans (in METERS!)
coords = [[4639.9e-6, -4571.7e-6, 1874.5e-6],
         [11117.6e-6, -6430.7e-6, 1870.7e-6]]

# the current global stage coordinates (in METERS!)
current_coords = [4639.9e-6, -4571.7e-6, 1874.5e-6]

# for each coordinate: where to save results
outpaths = ['C:/Users//RESOLFT/Desktop/AUTOMATION/12th_shipment_20170323/K562_HS2_Delta_HBG2/K562_HS2Delta_B_2/raw/',
           'C:/Users//RESOLFT/Desktop/AUTOMATION/12th_shipment_20170323/K562_HS2_Delta_HBG2/K562_wt_B_2/raw/']

# the maximum number of sted images to acquire
max_sted_images = 600

# the maximum time to spend in one area (in SECONDS)
max_time = 11.5 * 60 * 60 # 11hrs

## do the runs
do_runs(outpaths, coords, current_coords, max_time, max_sted_images, 
        sted_settings = 'C:/Users//RESOLFT/Desktop/det30sted.json')

# code below here is only for testing purposes

In [ ]:
im = Imspector()
ms = im.active_measurement()
stack1 = ms.stack(0).data()[0,:,:,:]
stack2 = ms.stack(1).data()[0,:,:,:]
stack1 = np.array(stack1, np.float)
stack2 = np.array(stack2, np.float)

sites = pair_finder_inner(stack1, stack2, 3, threshold = 0.8, invertAxes=True, normalize=False)
draw_detections_2c(stack1, stack2, [s[-1::-1] for s in sites], [1, 10], 0)
print(sites)


In [ ]:
current_coords = [-221.3e-6, -6300.2e-6, 1878.9e-6]
target = [-3924.1e-6, -3647.5e-6, 1889.7e-6]
actual_current = [2695.3e-6, -4124.7e-6, 1883.5e-6]

[2 * target[i] - current_coords[i] for i in range(len(target))]

In [ ]:
os.sep

In [ ]:
-0.0046678 - 0.0106896

In [ ]:
pair_finder(Imspector().active_measurement())